<a href="https://colab.research.google.com/github/juliaOdias/tcc-segmentacao-tumores/blob/main/notebooks/Tcc2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ETAPA 1 - IMPORTAR DADOS DO KAGGLE (DATASET BRISC correto)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Baixar o dataset com o slug correto
!kaggle datasets download -d briscdataset/brisc2025

# Após download, extrai o ZIP para a pasta 'brisc_data'
!unzip -q brisc2025.zip -d brisc_data

# Verifica os arquivos extraídos
import os
print("Conteúdo da pasta brisc_data:")
print(os.listdir("brisc_data")[:10])


Dataset URL: https://www.kaggle.com/datasets/briscdataset/brisc2025
License(s): Attribution 4.0 International (CC BY 4.0)
User cancelled operation
^C
replace brisc_data/brisc2025/classification_task/test/glioma/brisc2025_test_00001_gl_ax_t1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2  # OpenCV para leitura e processamento de imagens

# Caminhos para as imagens e máscaras
path_images = "brisc_data/brisc2025/segmentation_task/train/images"
path_masks  = "brisc_data/brisc2025/segmentation_task/train/masks"

# Listar os nomes dos arquivos
image_files = sorted(os.listdir(path_images))
mask_files  = sorted(os.listdir(path_masks))

print(f"Total de imagens de treino: {len(image_files)}")
print(f"Total de máscaras de treino: {len(mask_files)}")

# Ver os primeiros nomes
print("\nExemplos de nomes de arquivos:")
print("Imagem:", image_files[0])
print("Máscara:", mask_files[0])

# Função para visualizar imagem + máscara lado a lado
def visualizar_exemplo(idx):
    img_path = os.path.join(path_images, image_files[idx])
    mask_path = os.path.join(path_masks, mask_files[idx])

    # Lê a imagem e a máscara
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Plotar
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title("Imagem MRI")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap='gray')
    plt.title("Máscara (tumor)")
    plt.axis('off')

    plt.suptitle(f"Exemplo #{idx}", fontsize=14)
    plt.show()

# Visualizar um exemplo qualquer
visualizar_exemplo(0)


In [3]:
import numpy as np
from tqdm import tqdm  # barra de progresso

# Tamanho alvo
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Listas para armazenar imagens e máscaras processadas
X = []
y = []

# Pré-processar imagens
for img_file, mask_file in tqdm(zip(image_files, mask_files), total=len(image_files), desc="Processando imagens"):
    img_path = os.path.join(path_images, img_file)
    mask_path = os.path.join(path_masks, mask_file)

    # Lê imagem e máscara em grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Redimensiona
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

    # Normaliza para 0–1
    img = img / 255.0
    mask = mask / 255.0

    # Expande dimensões para formato (H, W, 1)
    img = np.expand_dims(img, axis=-1)
    mask = np.expand_dims(mask, axis=-1)

    # Adiciona às listas
    X.append(img)
    y.append(mask)

# Converte para arrays numpy
X = np.array(X)
y = np.array(y)

print(f"\n✅ Formato final das imagens: {X.shape}")
print(f"✅ Formato final das máscaras: {y.shape}")


Processando imagens: 100%|██████████| 3933/3933 [00:04<00:00, 923.49it/s]



✅ Formato final das imagens: (3933, 128, 128, 1)
✅ Formato final das máscaras: (3933, 128, 128, 1)


In [4]:
import matplotlib.pyplot as plt
from google.colab import files
import os

# Pasta onde vamos salvar as imagens processadas
save_dir = "/content/brisc_processed"
os.makedirs(save_dir + "/images", exist_ok=True)
os.makedirs(save_dir + "/masks", exist_ok=True)

# Como as imagens X e y estão normalizadas e têm shape (N, H, W, 1), vamos tirar o canal extra
# e salvar como PNG (escala de cinza 0-255)

for i in range(len(X)):
    # Recuperar a imagem e máscara no formato 0-255 uint8 para salvar
    img = (X[i].squeeze() * 255).astype('uint8')
    mask = (y[i].squeeze() * 255).astype('uint8')

    # Caminho para salvar
    img_path = os.path.join(save_dir, "images", f"image_{i:04d}.png")
    mask_path = os.path.join(save_dir, "masks", f"mask_{i:04d}.png")

    # Salvar com matplotlib
    plt.imsave(img_path, img, cmap='gray')
    plt.imsave(mask_path, mask, cmap='gray')

print(f"✅ Salvo {len(X)} imagens e máscaras em '{save_dir}'")

# Depois que salvar, para facilitar o download de alguns arquivos (exemplo: 5 imagens e máscaras)
for i in range(5):
    files.download(os.path.join(save_dir, "images", f"image_{i:04d}.png"))
    files.download(os.path.join(save_dir, "masks", f"mask_{i:04d}.png"))


✅ Salvo 3933 imagens e máscaras em '/content/brisc_processed'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
from tensorflow.keras import layers, Model, Input

def unet_model(input_size=(128, 128, 1)):
    inputs = Input(input_size)

    # --- ENCODER ---
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # --- BOTTLENECK ---
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    # --- DECODER ---
    u5 = layers.UpSampling2D((2, 2))(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.UpSampling2D((2, 2))(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.UpSampling2D((2, 2))(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c7)

    # --- SAÍDA ---
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [6]:
from tensorflow.keras.optimizers import Adam

model = unet_model()

model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      2,320 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │      9,248 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │     36,928 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │    147,584 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 32, 32,    │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 192)              │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 32, 32,    │    110,656 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │     36,928 │ conv2d_8[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 64, 64,    │          0 │ conv2d_9[0][0]  

 Total params: 487,009 (1.86 MB)

 Trainable params: 487,009 (1.86 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Caminhos
IMAGE_DIR = "brisc_data/brisc2025/segmentation_task/train/images"
MASK_DIR  = "brisc_data/brisc2025/segmentation_task/train/masks"

# Parâmetros
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Carregar imagens
def load_data(image_dir, mask_dir):
    images = []
    masks = []

    image_files = sorted(os.listdir(image_dir))
    mask_files = sorted(os.listdir(mask_dir))

    for img_name, mask_name in zip(image_files, mask_files):
        img = load_img(os.path.join(image_dir, img_name), color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img) / 255.0

        mask = load_img(os.path.join(mask_dir, mask_name), color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
        mask = img_to_array(mask) / 255.0

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Carregar imagens e máscaras
X, y = load_data(IMAGE_DIR, MASK_DIR)

# Dividir em treino e validação (80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train size:", X_train.shape)
print("Val size:", X_val.shape)


Train size: (3146, 128, 128, 1)
Val size: (787, 128, 128, 1)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Parar cedo se a validação não melhorar
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Treinamento
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=16,
    callbacks=[early_stop]
)


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 128, 128, 1))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 128, 128, 1))
  warnings.warn(msg)


197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9807 - loss: 0.1434

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 128, 128, 1))
  warnings.warn(msg)


197/197 ━━━━━━━━━━━━━━━━━━━━ 314s 2s/step - accuracy: 0.9807 - loss: 0.1431 - val_accuracy: 0.9814 - val_loss: 0.0521
Epoch 2/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9814 - loss: 0.0498 - val_accuracy: 0.9858 - val_loss: 0.0416
Epoch 3/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - accuracy: 0.9864 - loss: 0.0377 - val_accuracy: 0.9863 - val_loss: 0.0377
Epoch 4/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9878 - loss: 0.0316 - val_accuracy: 0.9881 - val_loss: 0.0317
Epoch 5/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9896 - loss: 0.0249 - val_accuracy: 0.9890 - val_loss: 0.0256
Epoch 6/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.9896 - loss: 0.0236 - val_accuracy: 0.9889 - val_loss: 0.0246
Epoch 7/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - accuracy: 0.9905 - loss: 0.0198 - val_accuracy: 0.9899 - val_loss: 0.0250
Epoch 8/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9908 - loss: 0.0190 - val_accuracy: 0.990

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de perda (loss)
plt.plot(history.history['loss'], label='Treinamento')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda (Loss) durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Gráfico de acurácia
plt.plot(history.history['accuracy'], label='Treinamento')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Seleciona aleatoriamente 5 imagens de validação
indices = random.sample(range(len(x_val)), 5)

plt.figure(figsize=(15, 10))

for i, idx in enumerate(indices):
    # Imagem original
    plt.subplot(5, 3, i*3 + 1)
    plt.imshow(x_val[idx].squeeze(), cmap='gray')
    plt.title('Imagem Original')
    plt.axis('off')

    # Máscara verdadeira
    plt.subplot(5, 3, i*3 + 2)
    plt.imshow(y_val[idx].squeeze(), cmap='gray')
    plt.title('Máscara Verdadeira')
    plt.axis('off')

    # Máscara prevista
    prediction = model.predict(np.expand_dims(x_val[idx], axis=0))[0]
    plt.subplot(5, 3, i*3 + 3)
    plt.imshow(prediction.squeeze(), cmap='gray')
    plt.title('Máscara Prevista')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Seleciona aleatoriamente 5 imagens de validação
indices = random.sample(range(len(X_val)), 5)

plt.figure(figsize=(15, 10))

for i, idx in enumerate(indices):
    # Imagem original
    plt.subplot(5, 3, i*3 + 1)
    plt.imshow(X_val[idx].squeeze(), cmap='gray')
    plt.title('Imagem Original')
    plt.axis('off')

    # Máscara verdadeira
    plt.subplot(5, 3, i*3 + 2)
    plt.imshow(y_val[idx].squeeze(), cmap='gray')
    plt.title('Máscara Verdadeira')
    plt.axis('off')

    # Máscara prevista
    prediction = model.predict(np.expand_dims(X_val[idx], axis=0))[0]
    plt.subplot(5, 3, i*3 + 3)
    plt.imshow(prediction.squeeze(), cmap='gray')
    plt.title('Máscara Prevista')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

# Seleciona aleatoriamente 5 índices do conjunto de validação
indices = random.sample(range(len(X_val)), 5)

plt.figure(figsize=(15, 10))

for i, idx in enumerate(indices):
    img = X_val[idx]
    true_mask = y_val[idx]

    # O modelo espera o batch, então expandimos dimensão
    pred_mask = model.predict(np.expand_dims(img, axis=0))[0]

    # Se for segmentação, pode ser máscara binária, ajustar limiar
    pred_mask_bin = (pred_mask > 0.5).astype(np.uint8)

    # Plot imagem original
    plt.subplot(5, 3, i*3 + 1)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title('Imagem Original')
    plt.axis('off')

    # Plot máscara verdadeira
    plt.subplot(5, 3, i*3 + 2)
    plt.imshow(true_mask.squeeze(), cmap='gray')
    plt.title('Máscara Verdadeira')
    plt.axis('off')

    # Plot máscara prevista
    plt.subplot(5, 3, i*3 + 3)
    plt.imshow(pred_mask_bin.squeeze(), cmap='gray')
    plt.title('Máscara Prevista')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Função para transformar máscara 2D binária em vetor 1D
def mask_to_binary_vector(mask):
    return mask.flatten()

# Supondo que X_val e y_val já estejam definidos e preparados (numpy arrays)

y_true_all = []
y_pred_all = []

for i in range(len(X_val)):
    true_mask = y_val[i]
    # Binariza máscara verdadeira (limiar 0.5)
    true_mask_bin = (true_mask > 0.5).astype(np.uint8)

    # Previsão do modelo (shape esperado: 128x128x1, pode ajustar se diferente)
    pred_mask = model.predict(np.expand_dims(X_val[i], axis=0))[0]
    # Binariza previsão
    pred_mask_bin = (pred_mask > 0.5).astype(np.uint8)

    # Converte máscara 2D para vetor 1D e adiciona na lista geral
    y_true_all.extend(mask_to_binary_vector(true_mask_bin))
    y_pred_all.extend(mask_to_binary_vector(pred_mask_bin))

# Agora calcula as métricas
acc = accuracy_score(y_true_all, y_pred_all)
prec = precision_score(y_true_all, y_pred_all)
rec = recall_score(y_true_all, y_pred_all)
f1 = f1_score(y_true_all, y_pred_all)

print(f'Accuracy: {acc:.4f}')
print(f'Precision: {prec:.4f}')
print(f'Recall: {rec:.4f}')
print(f'F1-score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import files

# Criar pasta para salvar gráficos
output_dir = "/content/outputs_tcc"
os.makedirs(output_dir, exist_ok=True)

# 1. Gráfico de perda
plt.figure()
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, "grafico_loss.png"))

# 2. Gráfico de acurácia
plt.figure()
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, "grafico_accuracy.png"))

# 3. Comparação de máscaras
indices = random.sample(range(len(X_val)), 5)
for i, idx in enumerate(indices):
    img = X_val[idx]
    true_mask = y_val[idx]
    pred_mask = model.predict(np.expand_dims(img, axis=0))[0]
    pred_mask_bin = (pred_mask > 0.5).astype(np.uint8)

    plt.figure(figsize=(9, 3))
    plt.subplot(1, 3, 1)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title("Imagem Original")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask.squeeze(), cmap='gray')
    plt.title("Máscara Real")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask_bin.squeeze(), cmap='gray')
    plt.title("Máscara Prevista")
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"comparacao_mascara_{i+1}.png"))
    plt.close()

# 4. Métricas em gráfico de barras
y_true_all = []
y_pred_all = []

for i in range(len(X_val)):
    true_mask = y_val[i]
    true_mask_bin = (true_mask > 0.5).astype(np.uint8)
    pred_mask = model.predict(np.expand_dims(X_val[i], axis=0))[0]
    pred_mask_bin = (pred_mask > 0.5).astype(np.uint8)
    y_true_all.extend(true_mask_bin.flatten())
    y_pred_all.extend(pred_mask_bin.flatten())

# Calcular
acc = accuracy_score(y_true_all, y_pred_all)
prec = precision_score(y_true_all, y_pred_all)
rec = recall_score(y_true_all, y_pred_all)
f1 = f1_score(y_true_all, y_pred_all)

# Salvar gráfico de barras com métricas
plt.figure()
plt.bar(["Accuracy", "Precision", "Recall", "F1-score"], [acc, prec, rec, f1], color="skyblue")
plt.ylim(0, 1)
plt.title("Métricas de Avaliação")
for i, v in enumerate([acc, prec, rec, f1]):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center')
plt.savefig(os.path.join(output_dir, "metricas_avaliacao.png"))
plt.close()


In [ ]:
# Baixar todos os arquivos gerados
for fname in sorted(os.listdir(output_dir)):
    files.download(os.path.join(output_dir, fname))
